In [267]:
import pandas as pd
import selenium
import webdriver_manager
from datetime import datetime 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support import expected_conditions as EC 
from time import sleep
import pandas as pd
from selenium import webdriver

from tqdm import tqdm
import os
from bs4 import BeautifulSoup
import requests
import re

import zipfile
import shutil

import os


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

import json
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()

import psycopg2
from psycopg2 import sql
pd.set_option('display.max_columns', None)

import sys
sys.path.append("../")  

In [257]:
keywallapop= os.getenv("keywallapop")

Cuidado, de wallapop solo hay 20 llamadas mensuales

In [258]:

url = "https://wallapop3.p.rapidapi.com/search"

querystring = {"query":"nissan","minPrice":"2000","sort":"mostRecent"}

headers = {
	"x-rapidapi-key": keywallapop,
	"x-rapidapi-host": "wallapop3.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

resposa=response.json()

SOLO NOS INTESESA:  
- id (pasaré a numérico)  
- user_id  
- title  
- description  
- category_id (lo quiero para filtrar por 100(codigo correspondiente a coches)para que no me coja otros códigos que son de carroceria etc)  
- price (que está en euros)  
- location (por si las moscas)  
- capacidad de traslado (no creo que me lo quede)  
- refurbished(igual la mantengo pero los true los paso a 1 y los false a 0 porque en sql existe el bit)

In [259]:
df_crudo=pd.DataFrame(resposa).head(4)

In [ ]:
with open("../src", "w") as json_file:
    json.dump(resposa, json_file, indent=4)

In [262]:
len (resposa)

40

In [263]:
lista=[]
df_final=pd.DataFrame()
for i in range(0,40):
    id_wallapop=resposa[i]["id"]
    id_user=resposa[i]["user_id"]
    title=resposa[i]["title"]
    descripcion=resposa[i]["description"]
    id_category=resposa[i]["category_id"]
    precio=resposa[i]["price"]["amount"]
    latitud=resposa[i]["location"]["latitude"]
    longitud=resposa[i]["location"]["longitude"]
    ciudad=resposa[i]["location"]["city"]
    provincia=resposa[i]["location"]["region2"]
    envio_disponible=resposa[i]["shipping"]["item_is_shippable"]
    reparado=resposa[i]["is_refurbished"]["flag"]

    lista.append([id_wallapop,id_user,title,descripcion,id_category,precio,latitud, longitud,ciudad,provincia,envio_disponible,reparado] )
    df=pd.DataFrame(lista)
    df_final= pd.concat([df_final,df])

In [264]:
df_crudo.head()
dicc_encabezados={0:"id_wallapop",
                  1: "id_user",
                  2: "title",
                  3: "descripcion",
                  4: "id_category",
                  5: "precio",
                  6: "latitud",
                  7: "longitud",
                  8: "ciudad",
                  9: "provincia",
                  10: "envio_disponible",
                  11: "reparado"}

In [265]:
df_final.columns= df_final.columns.map(dicc_encabezados)

In [270]:
df_final.sample(3)

,id_wallapop,id_user,title,descripcion,id_category,precio,latitud,longitud,ciudad,provincia,envio_disponible,reparado
15,8j34k1ny8769,8j3y83qwm169,Nissan Qashqai E-POWER N-Connecta Auto 140 kW ...,Precio oferta sujeto a financiación: 25863 EUR...,100,25863,38.976904,-3.929069,Ciudad Real,Ciudad Real,False,False
10,nz041kqvdejo,xzo871ykp469,Nissan Qashqai DIG-T 140 Acenta 4x2 103 kW (14...,Precio oferta sujeto a financiación: 15636 EUR...,100,15636,43.302337,-2.957724,Erandiogoikoa,Vizcaya,False,False
28,vjrqmg7kk9zk,xzo8xkly7l69,Nissan Qashqai dCi 115 N-Connecta 85 kW (115 CV),Precio oferta sujeto a financiación: 18172 EUR...,100,18172,36.874801,-2.441965,Huércal de Almería,Almería,False,False


In [41]:
df_final.shape

(820, 12)

ESTA LISTO PARA LA LIMPIEZA LO DE WALLAPOP

In [28]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 820 entries, 0 to 39
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_wallapop       820 non-null    object 
 1   id_user           820 non-null    object 
 2   title             820 non-null    object 
 3   descripcion       820 non-null    object 
 4   id_category       820 non-null    int64  
 5   precio            820 non-null    float64
 6   latitud           820 non-null    float64
 7   longitud          820 non-null    float64
 8   ciudad            820 non-null    object 
 9   provincia         820 non-null    object 
 10  envio_disponible  820 non-null    bool   
 11  reparado          820 non-null    bool   
dtypes: bool(2), float64(3), int64(1), object(6)
memory usage: 72.1+ KB


API seguridad del modelo

In [27]:
url_anio = f"https://api.nhtsa.gov/SafetyRatings/modelyear/2013/make/nissan"
response3 = requests.get(url_anio)
if response3.status_code == 200:
    modelos = [diccio["Model"] for diccio in response3.json().get("Results", [])]
else:
    print(f"Error: {response3.status_code} - {response3.text}")
    


['370Z',
 '370Z ROADSTER',
 'ALTIMA',
 'ARMADA',
 'CUBE',
 'FRONTIER',
 'GT-R',
 'JUKE',
 'LEAF',
 'MAXIMA',
 'MURANO',
 'MURANO CROSS',
 'NV1500',
 'NV200',
 'NV2500',
 'NV3500',
 'PATHFINDER',
 'QUEST',
 'ROGUE',
 'SENTRA',
 'TITAN',
 'VERSA',
 'XTERRA']

In [34]:


df_ratings = pd.DataFrame()

for anio in range(2011, 2026):
    url_anio = f"https://api.nhtsa.gov/SafetyRatings/modelyear/{anio}/make/nissan"
    response3 = requests.get(url_anio)
    
    # Verifica si la respuesta es exitosa antes de intentar interpretarla
    if response3.status_code == 200:
        try:
            modelos = [diccio["Model"] for diccio in response3.json().get("Results", [])]
        except (KeyError, ValueError) as e:
            print(f"Error al obtener modelos para el año {anio}: {e}")
            continue
    else:
        print(f"Error en la solicitud para el año {anio}: {response3.status_code}")
        continue

    modelos_ids = []
    for modelo in modelos:
        url3 = f"https://api.nhtsa.gov/SafetyRatings/modelyear/{anio}/make/nissan/model/{modelo}"
        response = requests.get(url3)

        if response.status_code == 200:
            try:
                modelos_ids.extend([diccio["VehicleId"] for diccio in response.json().get("Results", [])])
            except (KeyError, ValueError) as e:
                print(f"Error al obtener VehicleId para modelo {modelo} en el año {anio}: {e}")
                continue
        else:
            print(f"Error en la solicitud para modelo {modelo}, año {anio}: {response.status_code}")
            continue

    # Función para conseguir ratings con modelos_ids
    for modelo_id in modelos_ids:
        url2 = f"https://api.nhtsa.gov/SafetyRatings/VehicleId/{modelo_id}"
        response = requests.get(url2)

        if response.status_code == 200:
            try:
                df = pd.DataFrame(response.json().get("Results", []))
                df_ratings = pd.concat([df_ratings, df], ignore_index=True)
            except (KeyError, ValueError) as e:
                print(f"Error al obtener ratings para VehicleId {modelo_id}: {e}")
        else:
            print(f"Error en la solicitud para VehicleId {modelo_id}: {response.status_code}")


Error en la solicitud para el año 2011: 403
Error en la solicitud para el año 2012: 403
Error en la solicitud para el año 2013: 403
Error en la solicitud para el año 2014: 403
Error en la solicitud para el año 2015: 403
Error en la solicitud para el año 2016: 403
Error en la solicitud para el año 2017: 403
Error en la solicitud para el año 2018: 403
Error en la solicitud para el año 2019: 403
Error en la solicitud para el año 2020: 403
Error en la solicitud para el año 2021: 403
Error en la solicitud para el año 2022: 403
Error en la solicitud para el año 2023: 403
Error en la solicitud para el año 2024: 403
Error en la solicitud para el año 2025: 403


In [ ]:
df_ratings.reset_index(drop=True)

""


In [ ]:
df_ratings.sample(4)

,VehiclePicture,OverallRating,OverallFrontCrashRating,FrontCrashDriversideRating,FrontCrashPassengersideRating,OverallSideCrashRating,SideCrashDriversideRating,SideCrashPassengersideRating,combinedSideBarrierAndPoleRating-Front,combinedSideBarrierAndPoleRating-Rear,sideBarrierRating-Overall,RolloverRating,RolloverRating2,RolloverPossibility,RolloverPossibility2,dynamicTipResult,SidePoleCrashRating,NHTSAElectronicStabilityControl,NHTSAForwardCollisionWarning,NHTSALaneDepartureWarning,ComplaintsCount,RecallsCount,InvestigationCount,ModelYear,Make,Model,VehicleDescription,VehicleId,FrontCrashPicture,FrontCrashVideo,SideCrashPicture,SideCrashVideo,SidePolePicture,SidePoleVideo
0,https://static.nhtsa.gov/images/vehicles/8258_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,0.0,,Not Rated,Standard,No,No,17,0,1,2013,NISSAN,370Z,2013 Nissan 370Z 2 DR RWD,7558,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8258_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,0.000,0.0,,Not Rated,Standard,No,No,17,0,1,2013,NISSAN,370Z ROADSTER,2013 Nissan 370Z Roadster C RWD,7559,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8175_...,5,5,5,4,5,4,5,4,5,5,4,Not Rated,0.103,0.0,No Tip,5,Standard,No,No,2263,12,5,2013,NISSAN,ALTIMA,2013 Nissan Altima 4 DR FWD,7745,https://static.nhtsa.gov/crashTest/images/2013...,https://static.nhtsa.gov/crashTest/videos/2013...,https://static.nhtsa.gov/crashTest/images/2013...,https://static.nhtsa.gov/crashTest/videos/2013...,https://static.nhtsa.gov/crashTest/images/2013...,https://static.nhtsa.gov/crashTest/videos/2013...
0,https://static.nhtsa.gov/images/vehicles/8559_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,5,Not Rated,0.093,0.0,No Tip,Not Rated,Standard,No,No,2263,12,5,2013,NISSAN,ALTIMA,2013 Nissan Altima 2 DR FWD,7266,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8701_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,3,Not Rated,0.212,0.0,No Tip,Not Rated,Standard,No,No,14,2,0,2013,NISSAN,ARMADA,2013 Nissan Armada SUV RWD,7278,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8701_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,3,Not Rated,0.219,0.0,No Tip,Not Rated,Standard,No,No,14,2,0,2013,NISSAN,ARMADA,2013 Nissan Armada SUV 4WD,7277,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8728_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,4,Not Rated,0.174,0.0,No Tip,Not Rated,Standard,No,No,3,1,1,2013,NISSAN,CUBE,2013 Nissan Cube VAN FWD,7279,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8809_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,3,Not Rated,0.212,0.0,No Tip,Not Rated,Standard,No,No,46,1,0,2013,NISSAN,FRONTIER,2013 Nissan Frontier Crew Cab PU/CC RWD,7272,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8809_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,4,Not Rated,0.198,0.0,No Tip,Not Rated,Standard,No,No,46,1,0,2013,NISSAN,FRONTIER,2013 Nissan Frontier Crew Cab PU/CC 4WD,7271,NaN,NaN,NaN,NaN,NaN,NaN
0,https://static.nhtsa.gov/images/vehicles/8836_...,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,Not Rated,3,Not Rated,0.212,0.0,No Tip,Not Rated,Standard,No,No,46,1,0,2013,NISSAN,FRONTIER,2013 Nissan Frontier King Cab PU/EC RWD,7270,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ratings.info()

NameError: name 'np' is not defined

ESTÁ listo para la limpieza lo de SEGURIDAD

Empezamos con web scrapping de Quecochemecompro

In [59]:
url_nuevo = "https://www.quecochemecompro.com/"

driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_nuevo)

#acepto cookies
driver.find_element("css selector", "#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-y6krop").click()
sleep(2)
#me meto en nuevos
driver.find_element("css selector", "#header-menu > li:nth-child(1) > a").click()


sleep(2)
#click en el buscador 
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form").click()
sleep(2)
#meto nissan en el buscador
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form > input").send_keys("nissan", Keys.ENTER)

#saco los nombres para meterlos en una lista
#driver.execute_script("window.scrollBy(0, 200);")

sopa_nuevos = BeautifulSoup(driver.page_source, 'html.parser')
lista_nombre_modelo=[]
for modelo in sopa_nuevos.find_all("li", {"class":"mx-auto w-full min-w-[20rem]"}):
    tag_a = modelo.find("a", title=True)
    if tag_a:
        lista_nombre_modelo.append(tag_a["title"])

#me meto en cada uno de los modelos para sacar la info
for n_modelo in range(1,len(lista_nombre_modelo)+1):
    sleep(2)
    driver.find_element("css selector", f"#results > ul > li:nth-child({n_modelo}) > div > div.relative.z-10 > div.flex.justify-between.items-end.py-3.leading-5.text-17 > div:nth-child(1) > a").click()
    sopa_tabla = BeautifulSoup(driver.page_source, 'html.parser')
    tabla=sopa_tabla.find_all("table")
    
    
    # driver.back()
    

# url_par_tabla=f"https://www.quecochemecompro.com/precios/{}/"





NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#results > ul > li:nth-child(2) > div > div.relative.z-10 > div.flex.justify-between.items-end.py-3.leading-5.text-17 > div:nth-child(1) > a"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF770B03AF5+28005]
	(No symbol) [0x00007FF770A683F0]
	(No symbol) [0x00007FF77090580A]
	(No symbol) [0x00007FF770955A3E]
	(No symbol) [0x00007FF770955D2C]
	(No symbol) [0x00007FF77099EA97]
	(No symbol) [0x00007FF77097BA7F]
	(No symbol) [0x00007FF77099B8B3]
	(No symbol) [0x00007FF77097B7E3]
	(No symbol) [0x00007FF7709475C8]
	(No symbol) [0x00007FF770948731]
	GetHandleVerifier [0x00007FF770DF646D+3118813]
	GetHandleVerifier [0x00007FF770E46CC0+3448624]
	GetHandleVerifier [0x00007FF770E3CF3D+3408301]
	GetHandleVerifie

In [67]:
tablita=tabla[0]
lista_encabezados = tablita.findAll("thead")

In [71]:
lista_encabezados[0].getText()


'\n\n\n          Versión\n        \n\n          Etiqueta\n        \n\n          Puertas\n        \n\n          CV\n        \n\n          Consumo\n        \n\n          Motor\n        \n\n            PVP\n          \n\n\n              Oferta\n              \n\n\n\n'

In [72]:

lista_filas= tablita.findAll("tbody")

In [79]:
# for i, fila in enumerate(lista_filas):
#     print(f"La fila {i + 1} es: {fila.getText()}")
filas_completas = []

# Iniciamos un bucle 'for' para iterar a través de la lista_filas_ibex a partir de la segunda fila
for fila in lista_filas:
    fila_texto = fila.text
    elementos_fila = fila_texto.split("\n\n")

    filas_completas.append(elementos_fila)

In [90]:
for elemento in filas_completas:
    elemento=elemento[0].drop("")

AttributeError: 'str' object has no attribute 'drop'

In [ ]:
url_nuevo = "https://www.quecochemecompro.com/"

driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_nuevo)

#acepto cookies
driver.find_element("css selector", "#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-y6krop").click()
sleep(2)
#me meto en nuevos
driver.find_element("css selector", "#header-menu > li:nth-child(1) > a").click()


sleep(2)
#click en el buscador 
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form").click()
sleep(2)
#meto nissan en el buscador
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form > input").send_keys("nissan", Keys.ENTER)

#saco los nombres para meterlos en una lista

sopa_nuevos = BeautifulSoup(driver.page_source, 'html.parser')
lista_nombre_modelo=[]
for modelo in sopa_nuevos.find_all("li", {"class":"mx-auto w-full min-w-[20rem]"}):
    tag_a = modelo.find("a", title=True)
    if tag_a:
        lista_nombre_modelo.append(tag_a["title"])


df_datos_modelos = pd.DataFrame()
for n_modelo in range(1, len(lista_nombre_modelo) + 1):
    sleep(2)
    # Hacer click en el modelo específico
    driver.find_element("css selector", f"#results > ul > li:nth-child({n_modelo}) > div > div.relative.z-10 > div.flex.justify-between.items-end.py-3.leading-5.text-17 > div:nth-child(1) > a").click()
    # sopa_tabla = BeautifulSoup(driver.page_source, 'html.parser')
    sleep(2)
    df_tablas=pd.DataFrame()
    try:  # Encontrar la tabla de datos
        sopa_tabla = BeautifulSoup(driver.page_source, 'html.parser')
        for i in range(2,9): #hace click por si hay más tablas

            tabla = sopa_tabla.find("table")  #a lo mejor no me coge en algunas porque hay otra tabla
            
            
            try:
                driver.find_element("css selector", f"#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child({i}) > a").click()
                sleep(2)
                
                
                encabezados = [th.get_text(strip=True) for th in tabla.find_all("thead")[0].find_all("th")]

                filas = []
                for tr in tabla.find_all("tbody")[0].find_all("tr"):
                    filas.append([td.get_text(strip=True) for td in tr.find_all("td")])

                df_tabla = pd.DataFrame(filas, columns=encabezados)
                df_tabla["Modelo"] = lista_nombre_modelo[n_modelo - 1]
                df_tablas= pd.concat([df_tablas, df_tabla])
        


            except:
                break

    except:
        driver.back()
        continue
        

#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child(2) > a
#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child(3) > a
#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child(4) > a

        
    df_datos_modelos= pd.concat([df_datos_modelos, df_tablas], ignore_index=True)
    driver.back()    




In [160]:
df_datos_modelos

,Versión,Etiqueta,Puertas,CV,Consumo,Motor,PVP,Oferta,Modelo,Autonomía
0,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,,5,140,"6,3 l/100Km",MHEV Gasolina,31.200 €,Pedir oferta,Nissan Qashqai,NaN
1,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,,5,140,"6,3 l/100Km",MHEV Gasolina,32.700 €,Pedir oferta,Nissan Qashqai,NaN
2,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) AC...,,5,158,"6,3 l/100Km",MHEV Gasolina,34.300 €,Pedir oferta,Nissan Qashqai,NaN
3,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) N-...,,5,140,"6,4 l/100Km",MHEV Gasolina,34.850 €,Pedir oferta,Nissan Qashqai,NaN
4,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,,5,140,"6,4 l/100Km",MHEV Gasolina,35.000 €,Pedir oferta,Nissan Qashqai,NaN
...,...,...,...,...,...,...,...,...,...,...
4151,NISSAN PRIMASTAR 2.0DCI 81KW (110CV) COMFORT L...,,4,110,"7,0 l/100Km",Diesel,43.540 €,Pedir oferta,Nissan NV300,NaN
4152,NISSAN GT-R 3.8 V6 419KW (570CV) PRESTIGE,,2,570,"12,5 l/100Km",Gasolina,111.290 €,Pedir oferta,Nissan GT-R,NaN
4153,NISSAN GT-R 3.8 V6 419KW (570CV) BLACK EDITION,,2,570,"12,5 l/100Km",Gasolina,114.290 €,Pedir oferta,Nissan GT-R,NaN
4154,NISSAN GT-R 3.8 V6 419KW (570CV) TRACK EDITION...,,2,570,"12,5 l/100Km",Gasolina,127.290 €,Pedir oferta,Nissan GT-R,NaN


Por aquí chat porque no sé porque no me coge las tablas varias de cada modelo y encima me genera duplicados

In [175]:
url_nuevo = "https://www.quecochemecompro.com/"

driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_nuevo)

#acepto cookies
driver.find_element("css selector", "#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-y6krop").click()
sleep(2)
#me meto en nuevos
driver.find_element("css selector", "#header-menu > li:nth-child(1) > a").click()


sleep(2)
#click en el buscador 
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form").click()
sleep(2)
#meto nissan en el buscador
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form > input").send_keys("nissan", Keys.ENTER)

#saco los nombres para meterlos en una lista

sopa_nuevos = BeautifulSoup(driver.page_source, 'html.parser')
lista_nombre_modelo=[]
for modelo in sopa_nuevos.find_all("li", {"class":"mx-auto w-full min-w-[20rem]"}):
    tag_a = modelo.find("a", title=True)
    if tag_a:
        lista_nombre_modelo.append(tag_a["title"])
df_datos_modelos= pd.DataFrame()
df_tablas = pd.DataFrame()
for n_modelo in range(1, len(lista_nombre_modelo) + 1):
    sleep(2)
    
    # Hacer click en el modelo específico
    try:
         driver.find_element("css selector", f"#results > ul > li:nth-child({n_modelo}) > div > div.relative.z-10 > div.flex.justify-between.items-end.py-3.leading-5.text-17 > div:nth-child(1) > a").click()
    
    except Exception as e:
        print(f"Error al hacer click en el modelo {n_modelo}: {e}")
        continue  # Continuar con el siguiente modelo

    

    
    try:  # Intentar encontrar la tabla de datos
        
        for i in range(2, 9):  # Hacer click por si hay más tablas
            try:

                sopa_tabla = BeautifulSoup(driver.page_source, 'html.parser')
                # Hacer click para cargar más versiones
                try:
                    driver.find_element("css selector", f"#versions > div > div:nth-child(2) > table > tfoot > tr > td > ul > li:nth-child({i}) > a").click()

                except:
                    # Si no encuentra el selector con nth-child(2), intenta con nth-child(3)
                    sleep(2)
                    driver.find_element("css selector", f"#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child({i}) > a").click()
                sleep(2)
                tabla = sopa_tabla.find("table") 
                
                encabezados = [th.get_text(strip=True) for th in tabla.find_all("thead")[0].find_all("th")]
                filas = []
                for tr in tabla.find_all("tbody")[0].find_all("tr"):
                    filas.append([td.get_text(strip=True) for td in tr.find_all("td")])

                df_tabla = pd.DataFrame(filas, columns=encabezados)
                df_tabla["Modelo"] = lista_nombre_modelo[n_modelo - 1]
                df_tablas = pd.concat([df_tablas, df_tabla])
            except Exception as e:
                print(f"Error al procesar la tabla para el modelo {n_modelo}, versión {i}: {e}")
                break  # Salir del bucle for i si hay un error
    except Exception as e:
        print(f"Error al encontrar la tabla para el modelo {n_modelo}: {e}")
        driver.back()  # Regresar a la página anterior si no se puede encontrar la tabla
        continue  # Continuar con el siguiente modelo
    df_datos_modelos= pd.concat([df_datos_modelos, df_tablas], ignore_index=True)
    # Procesar df_tablas o hacer algo con los datos recogidos
      # Mostrar los datos recogidos
    driver.back()  # Regresar a la lista de modelos
    print("llegué al back")

# Cerrar el driver al finalizar



Error al procesar la tabla para el modelo 1, versión 6: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child(6) > a"}
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF770B03AF5+28005]
	(No symbol) [0x00007FF770A683F0]
	(No symbol) [0x00007FF77090580A]
	(No symbol) [0x00007FF770955A3E]
	(No symbol) [0x00007FF770955D2C]
	(No symbol) [0x00007FF77099EA97]
	(No symbol) [0x00007FF77097BA7F]
	(No symbol) [0x00007FF77099B8B3]
	(No symbol) [0x00007FF77097B7E3]
	(No symbol) [0x00007FF7709475C8]
	(No symbol) [0x00007FF770948731]
	GetHandleVerifier [0x00007FF770DF646D+3118813]
	GetHandleVerifier [0x00007FF770E46CC0+3448624]
	GetHandleVerifier [0x00007FF770E3CF3D+3408301]
	GetHandleVerifier [0x00007FF770BCA44B

In [ ]:
df_datos_modelos

,Versión,Etiqueta,Puertas,CV,Consumo,Motor,PVP,Oferta,Modelo,Autonomía
0,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,,5,140,"6,3 l/100Km",MHEV Gasolina,31.200 €,Pedir oferta,Nissan Qashqai,NaN
1,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,,5,140,"6,3 l/100Km",MHEV Gasolina,32.700 €,Pedir oferta,Nissan Qashqai,NaN
2,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) AC...,,5,158,"6,3 l/100Km",MHEV Gasolina,34.300 €,Pedir oferta,Nissan Qashqai,NaN
3,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) N-...,,5,140,"6,4 l/100Km",MHEV Gasolina,34.850 €,Pedir oferta,Nissan Qashqai,NaN
4,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,,5,140,"6,4 l/100Km",MHEV Gasolina,35.000 €,Pedir oferta,Nissan Qashqai,NaN
...,...,...,...,...,...,...,...,...,...,...
3576,NISSAN GT-R 3.8 V6 441KW (600CV) NISMO,,2,600,"12,5 l/100Km",Gasolina,220.290 €,Pedir oferta,Nissan GT-R,NaN
3577,NISSAN GT-R 3.8 V6 419KW (570CV) PRESTIGE,,2,570,"12,5 l/100Km",Gasolina,111.290 €,Pedir oferta,Nissan GT-R,NaN
3578,NISSAN GT-R 3.8 V6 419KW (570CV) BLACK EDITION,,2,570,"12,5 l/100Km",Gasolina,114.290 €,Pedir oferta,Nissan GT-R,NaN
3579,NISSAN GT-R 3.8 V6 419KW (570CV) TRACK EDITION...,,2,570,"12,5 l/100Km",Gasolina,127.290 €,Pedir oferta,Nissan GT-R,NaN


In [179]:
df_datos_modelos.duplicated().sum()

np.int64(3361)

In [180]:
df_datos_modelos[df_datos_modelos["Versión"]=="Nissan Qashqai 1.3 DIG-T MHEV 116KW TEKNA PREMIUM"]

,Versión,Etiqueta,Puertas,CV,Consumo,Motor,PVP,Oferta,Modelo,Autonomía


In [129]:
df_datos_modelos=df_datos_modelos.reset_index(drop=True)

In [130]:
df_datos_modelos.to_csv("../datos/datos_modelos_nuevos")

In [139]:
df_datos_modelos_nuevos=pd.read_csv("../datos/datos_modelos_nuevos", index_col= "Unnamed: 0" )

In [148]:
df_datos_modelos_nuevos.head(5)

,Versión,Etiqueta,Puertas,CV,Consumo,Motor,PVP,Oferta,Modelo,Autonomía
0,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,NaN,5,140,"6,3 l/100Km",MHEV Gasolina,31.200 €,Pedir oferta,Nissan Qashqai,NaN
1,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,NaN,5,140,"6,3 l/100Km",MHEV Gasolina,32.700 €,Pedir oferta,Nissan Qashqai,NaN
2,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) AC...,NaN,5,158,"6,3 l/100Km",MHEV Gasolina,34.300 €,Pedir oferta,Nissan Qashqai,NaN
3,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) N-...,NaN,5,140,"6,4 l/100Km",MHEV Gasolina,34.850 €,Pedir oferta,Nissan Qashqai,NaN
4,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,NaN,5,140,"6,4 l/100Km",MHEV Gasolina,35.000 €,Pedir oferta,Nissan Qashqai,NaN


In [144]:
df_datos_modelos_nuevos[df_datos_modelos_nuevos.duplicated()]

,Versión,Etiqueta,Puertas,CV,Consumo,Motor,PVP,Oferta,Modelo,Autonomía
10,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,NaN,5,140,"6,3 l/100Km",MHEV Gasolina,31.200 €,Pedir oferta,Nissan Qashqai,NaN
11,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,NaN,5,140,"6,3 l/100Km",MHEV Gasolina,32.700 €,Pedir oferta,Nissan Qashqai,NaN
12,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) AC...,NaN,5,158,"6,3 l/100Km",MHEV Gasolina,34.300 €,Pedir oferta,Nissan Qashqai,NaN
13,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) N-...,NaN,5,140,"6,4 l/100Km",MHEV Gasolina,34.850 €,Pedir oferta,Nissan Qashqai,NaN
14,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,NaN,5,140,"6,4 l/100Km",MHEV Gasolina,35.000 €,Pedir oferta,Nissan Qashqai,NaN
...,...,...,...,...,...,...,...,...,...,...
271,NISSAN PRIMASTAR 2.0DCI 81KW (110CV) GO L2H1 C...,NaN,4,110,"7,6 l/100Km",Diesel,42.598 €,Pedir oferta,Nissan NV300,NaN
272,NISSAN PRIMASTAR 2.0DCI 110KW (150CV) GO L1H1 ...,NaN,4,150,"7,0 l/100Km",Diesel,42.840 €,Pedir oferta,Nissan NV300,NaN
273,NISSAN PRIMASTAR 2.0DCI 81KW (110CV) COMFORT L...,NaN,4,110,"7,0 l/100Km",Diesel,43.115 €,Pedir oferta,Nissan NV300,NaN
274,NISSAN PRIMASTAR 2.0DCI 110KW (150CV) GO L1H1 ...,NaN,4,150,"7,0 l/100Km",Diesel,43.265 €,Pedir oferta,Nissan NV300,NaN


Empezamos con web scrapping de Canal car

In [242]:
url_segunda_mano = "https://www.canalcar.es/"

driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_segunda_mano)

#acepto cookies
driver.find_element("css selector", "#CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll").click()
sleep(8)
#quito el popup
driver.find_element("css selector", "#sf-chat-tool > div > div > div > div.sf-push-input.sf-push-input-position-right.sf-popup-has-image.sf-push-align-center.sf-push-type-minimalText > div.sf-push-icon > svg").click()
sleep(2)

driver.execute_script("window.scrollTo(0,900)") 
sleep(2)
#pincho nissan
driver.find_element("css selector", "body > main > section:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div > ul > li:nth-child(8) > a").click()
sleep(8)

#quito el popup de si me interesa este
driver.find_element("css selector", "#sf-chat-tool > div > div > div > div.sf-push-input.sf-push-input-position-right.sf-popup-has-image.sf-push-align-center.sf-push-type-minimalText > div.sf-push-icon > svg").click()
sleep(2)

sopa_segunda = BeautifulSoup(driver.page_source, 'html.parser')

#conseguir nombre
lista_nombres = sopa_segunda.findAll("h3", {"class": "vehicle__title"})

#conseguir datos
lista_datos= sopa_segunda.findAll("ul", {"class":"vehicle__specs vehicle__features"})

#Conseguir precio
lista_precios= sopa_segunda.findAll("span", {"class":"vehicle__price"})


In [222]:
todos_nombres=[nombre.getText().strip().replace("                        1  ", " ").replace("                        0  ", " ") for nombre in lista_nombres]

In [272]:
todos_datos=[dato.getText().lower().replace(".","").replace("kms","").split() for dato in lista_datos]

In [247]:
todos_precios=[precio.getText().strip().replace("                    €","").replace(".","") for precio in lista_precios]

In [279]:
df_coche_segunda=pd.DataFrame()
df_nombre=pd.DataFrame(todos_nombres)
df_dato=pd.DataFrame(todos_datos)
df_precio=pd.DataFrame(todos_precios)
df_coche_segunda=pd.concat([df_coche_segunda,df_nombre,df_dato,df_precio], ignore_index=True, axis=1)

In [280]:
dicc_encabezados2={0:"title",
                  1: "anio",
                  2: "kilometraje",
                  3: "fuel",
                  4: "tipo",
                  5: "precio"}

In [281]:
df_coche_segunda.columns=df_coche_segunda.columns.map(dicc_encabezados2)

In [282]:
df_coche_segunda

,title,anio,kilometraje,fuel,tipo,precio
0,Nissan 370Z 3.7G Coupé GT 241 kW (328 CV),2020,16314,gasolina,manual,38840
1,Nissan X-Trail dCi 150 Tekna 4x2 110 kW (150 CV),2019,50964,diésel,manual,22240
2,Nissan Juke 1.6i N-Tec,2013,154942,gasolina,manual,9035
3,Nissan Qashqai DIG-T 140 Acenta 4x2 103 kW (14...,2019,37289,gasolina,manual,18640
4,Nissan Qashqai DIG-T 140 N-Connecta 103 kW (14...,2019,53458,gasolina,manual,17940
5,Nissan X-Trail dCi 150 N-Tec 110 kW (150 CV),2020,123086,diésel,manual,21140
6,Nissan Juke 1.5 dCi Acenta 4x2 81 kW (110 CV),2017,78472,diésel,manual,13590
7,Nissan X-Trail dCi 130 360 XTronic 96 kW (130 CV),2015,207755,diésel,automático,13885
8,Nissan e-NV200 Furgon Electrica Profesional 80...,2017,60693,eléctrico,automático,15840
9,Nissan Note 1.5dCi Naru Edition,2014,93248,diésel,manual,8535


Tengo ya el df de Canalcar (Falta el de quecochemecompro)